# ETLs

## Imports

In [423]:
import pandas as pd
import sqlite3
import pyodbc

select_tables = "SELECT name FROM sqlite_master WHERE type='table'"

sales_con = sqlite3.connect("go_sales.sqlite")
sales_tables = pd.read_sql_query(select_tables, sales_con)

sales_country       = pd.read_sql_query("SELECT * FROM country;", sales_con)
order_details       = pd.read_sql_query("SELECT * FROM order_details;", sales_con)
order_header        = pd.read_sql_query("SELECT * FROM order_header;", sales_con)
order_method        = pd.read_sql_query("SELECT * FROM order_method;", sales_con)
product             = pd.read_sql_query("SELECT * FROM product;", sales_con)
product_line        = pd.read_sql_query("SELECT * FROM product_line;", sales_con)
product_type        = pd.read_sql_query("SELECT * FROM product_type;", sales_con)
sales_retailer_site = pd.read_sql_query("SELECT * FROM retailer_site;", sales_con)
return_reason       = pd.read_sql_query("SELECT * FROM return_reason;", sales_con)
returned_item       = pd.read_sql_query("SELECT * FROM returned_item;", sales_con)
sales_branch        = pd.read_sql_query("SELECT * FROM sales_branch;", sales_con)
sales_staff         = pd.read_sql_query("SELECT * FROM sales_staff;", sales_con)
SALES_TARGETData    = pd.read_sql_query("SELECT * FROM SALES_TARGETData;", sales_con)
sqlite_sequence     = pd.read_sql_query("SELECT * FROM sqlite_sequence;", sales_con)
print("Imported sales tables")

staff_con = sqlite3.connect("go_staff.sqlite")
staff_tables = pd.read_sql_query(select_tables, staff_con)

course            = pd.read_sql_query("SELECT * FROM course;", staff_con)
sales_branch      = pd.read_sql_query("SELECT * FROM sales_branch;", staff_con)
sales_staff       = pd.read_sql_query("SELECT * FROM sales_staff;", staff_con)
satisfaction      = pd.read_sql_query("SELECT * FROM satisfaction;", staff_con)
satisfaction_type = pd.read_sql_query("SELECT * FROM satisfaction_type;", staff_con)
training          = pd.read_sql_query("SELECT * FROM training;", staff_con)
print("Imported staff tables")

crm_con = sqlite3.connect("go_crm.sqlite")
crm_tables = pd.read_sql_query(select_tables, crm_con)
                           
age_group             = pd.read_sql_query("SELECT * FROM age_group;", crm_con)
crm_country           = pd.read_sql_query("SELECT * FROM country;", crm_con)
retailer              = pd.read_sql_query("SELECT * FROM retailer;", crm_con)
retailer_contact      = pd.read_sql_query("SELECT * FROM retailer_contact;", crm_con)
retailer_headquarters = pd.read_sql_query("SELECT * FROM retailer_headquarters;", crm_con)
retailer_segment      = pd.read_sql_query("SELECT * FROM retailer_segment;", crm_con)
crm_retailer_site     = pd.read_sql_query("SELECT * FROM retailer_site;", crm_con)
retailer_type         = pd.read_sql_query("SELECT * FROM retailer_type;", crm_con)
sales_demographic     = pd.read_sql_query("SELECT * FROM sales_demographic;", crm_con)
sales_territory       = pd.read_sql_query("SELECT * FROM sales_territory;", crm_con)
print("Imported crm tables")

inventory_level = pd.read_csv("GO_SALES_INVENTORY_LEVELSData.csv")
print("Imported inventory table")

sales_forecast = pd.read_csv("GO_SALES_PRODUCT_FORECASTData.csv")
print("Imported sales product forecast table")

Imported sales tables
Imported staff tables
Imported crm tables
Imported inventory table
Imported sales product forecast table


## SQL Server connection

In [424]:
servername = 'DESKTOP-9F8A8PF\\MSSQLSERVER01'
database = 'Datawarehouse'

sql_server_conn = pyodbc.connect(f"DRIVER={{SQL Server}};SERVER={servername};DATABASE={database};Trusted_Connection=yes")
cursor = sql_server_conn.cursor()

try:
    cursor.execute("SELECT * FROM Test")
    test = cursor.fetchall()
    print(test)
except pyodbc.Error as e:
    print(e)

[(1, 'test      '), (2, 'test2     ')]


In [425]:
"""
Flexible method to merge two tables
- NaN values of one dataframe can be filled by the other dataframe
- Uses all available columns
- Errors when a row of the two dataframes doesn't match (df1 has 'A' and df2 has 'B' in row)
"""
def merge_tables(df1, df2, index_col):
    # Ensure 'CODE' is set as the index for both DataFrames
    if index_col not in df1.columns or index_col not in df2.columns:
        raise KeyError(f"{index_col} must be a column in both DataFrames.")
    
    df1 = df1.set_index(index_col)
    df2 = df2.set_index(index_col)

    # Identify common and exclusive columns
    common_columns = df1.columns.intersection(df2.columns)
    exclusive_df1 = df1.columns.difference(df2.columns)
    exclusive_df2 = df2.columns.difference(df1.columns)

    # Concatenate exclusive columns from each DataFrame onto the other
    df1_combined = pd.concat([df1, df2[exclusive_df2]], axis=1, sort=False)
    df2_combined = pd.concat([df2, df1[exclusive_df1]], axis=1, sort=False)

    # Resolve common columns with nulls and conflicts
    for col in common_columns:
        # Align the Series from both DataFrames for comparison
        series1, series2 = df1_combined[col].align(df2_combined[col])

        # Check for conflicts (non-null values that do not match)
        conflict_mask = (~series1.isnull() & ~series2.isnull() & (series1 != series2))
        if conflict_mask.any():
            raise ValueError(f"Merge failed due to conflict in column '{col}'")

        # Use values from df2 where df1 is null (prioritizing df1 values)
        df1_combined[col] = series1.combine_first(series2)

    return df1_combined

# Merge duplicate tables into single table
retailer_site = merge_tables(sales_retailer_site, crm_retailer_site, 'RETAILER_SITE_CODE')
# Column name mismatch
sales_country = sales_country.rename(columns={'COUNTRY': 'COUNTRY_EN'})
country = merge_tables(sales_country, crm_country, 'COUNTRY_CODE')

## Rename Dictionary

In [426]:
rename_mapping = {
    "PRODUCT_NAME": "PRODUCT_name",
    "PRODUCT_IMAGE": "PRODUCT_image",
    "PRODUCT_NUMBER": "PRODUCT_id",
    "DESCRIPTION": "PRODUCT_description",
    "LANGUAGE": "PRODUCT_LANGUAGE_name",
    "PRODUCTION_COST": "PRODUCT_PRODUCTION_COST_cost",
    "MARGIN": "PRODUCT_MARGIN_percentage",
    "INTRODUCTION_DATE": "PRODUCT_INTRODUCTION_DATE_date",
    "PRODUCT_LINE_CODE": "PRODUCT_LINE_code",
    "PRODUCT_LINE_EN": "PRODUCT_LINE_name",
    'SALES_STAFF_CODE': 'SALES_STAFF_code', 
    'FIRST_NAME': 'FIRST_NAME_name', 
    'LAST_NAME': 'LAST_NAME_name', 
    'POSITION_EN' : 'POSITION_name',
    'WORK_PHONE': 'WORK_PHONE_number', 
    'EXTENSION': 'EXTENSION_number', 
    'FAX': 'FAX_number', 
    'EMAIL': 'EMAIL_address', 
    'DATE_HIRED': 'DATE_HIRED_date',
    'SALES_BRANCH_CODE': 'SALES_BRANCH_code', 
    'MANAGER_CODE': "MANAGER_code", 
    'ADDRESS1': 'ADDRESS1_address',
    'ADDRESS2': 'ADDRESS2_address', 
    'CITY': 'CITY_name', 
    'REGION': 'REGION_name', 
    'POSTAL_ZONE': 'POSTAL_ZONE_code', 
    'COUNTRY_CODE': 'COUNTRY_code', 
    'COUNTRY_EN': 'COUNTRY_name', 
    'LANGUAGE' : 'LANGUAGE_name', 
    'CURRENCY_NAME': 'CURRENCY_name', 
    'FLAG_IMAGE': 'FLAG_image', 
    'SALES_TERRITORY_CODE': 'SALES_TERRITORY_code',
    'TERRITORY_NAME_EN': 'TERRITORY_name',
    'ORDER_NUMBER': 'ORDER_number', 
    'RETAILER_NAME': 'RETAILER_name',
    'RETAILER_CONTACT_CODE': 'RETAILER_CONTACT_code', 
    'SALES_STAFF_CODE': 'SALES_STAFF_code',
    'ORDER_DATE': 'ORDER_DATE_date', 
    'ORDER_METHOD_CODE': 'ORDER_METHOD_code', 
    'ORDER_METHOD_EN': 'ORDER_METHOD_name',
    'RETAILER_CONTACT_CODE' : 'RETAILER_CONTACT_code', 
    'RETAILER_SITE_CODE': 'RETAILER_SITE_code', 
    'FIRST_NAME' : 'FIRST_NAME_name',
    'LAST_NAME': 'LAST_NAME_name', 
    'JOB_POSITION_EN' : 'JOB_POSITION_EN',
    'EXTENSION': 'EXTENSION_number', 
    'FAX': 'FAX_number', 
    'E_MAIL': 'EMAIL_address', 
    'GENDER': 'GENDER_code', 
    'RETAILER_CODE': 'RETAILER_code',
    'CITY': 'CITY_name', 
    'REGION': 'REGION_name',
    'POSTAL_ZONE': 'POSTAL_ZONE_code', 
    'COUNTRY_CODE': 'COUNTRY_code', 
    'ACTIVE_INDICATOR': 'ACTIVE_INDICATOR_bool',
    'COUNTRY_EN': 'COUNTRY_name', 
    'LANGUAGE' : 'LANGUAGE_name', 
    'CURRENCY_NAME': 'CURRENCY_name', 
    'FLAG_IMAGE': 'FLAG_image', 
    'SALES_TERRITORY_CODE': 'SALES_TERRITORY_code',
    'TERRITORY_NAME_EN': 'TERRITORY_name',
    'SEGMENT_CODE': 'SEGMENT_code',
    'COMPANY_NAME': 'COMPANY_name',
    'RETAILER_TYPE_CODE': 'RETAILER_TYPE_code',
    'RETAILER_CODEMR': 'RETAILER_MR_code',
    'RETAILER_TYPE_EN': 'RETAILER_TYPE_name',
    'PHONE': 'PHONE_number',
    'EXPECTED_VOLUME': 'EXPECTED_VOLUME_volume',
    'YEAR': 'YEAR_number',
    'MONTH': 'MONTH_number',
    'SATISFACTION_TYPE_DESCRIPTION': 'SATISFACTION_TYPE_description',
    'COURSE_DESCRIPTION': 'COURSE_description',
    'RETURN_DESCRIPTION_EN': 'RETURN_REASON_description',
    'ORDER_DETAIL_CODE': 'ORDER_DETAIL_code', 
    'RETURN_CODE': 'RETURN_code', 
    'RETURN_DATE': 'RETURN_DATE_date', 
    'RETURN_QUANTITY': 'RETURN_QUANTITY_number', 
    'RETURN_REASON_CODE': 'RETURN_REASON_code', 
    'QUANTITY' : 'QUANTITY_number',
    'UNIT_COST' : 'UNIT_COST_dollar', 
    'UNIT_PRICE' : 'UNIT_PRICE_dollar', 
    'UNIT_SALE_PRICE' : 'UNIT_SALE_PRICE_dollar', 
}

## Product ETL

In [427]:
product_etl = pd.merge(product, product_type, on="PRODUCT_TYPE_CODE")
product_etl = pd.merge(product_etl, product_line, on="PRODUCT_LINE_CODE")\
    [[
        "PRODUCT_NAME", "PRODUCT_IMAGE", "PRODUCT_NUMBER",
        "DESCRIPTION", "LANGUAGE", "PRODUCTION_COST",
        "MARGIN", "INTRODUCTION_DATE", "PRODUCT_LINE_CODE",
        "PRODUCT_LINE_EN"
    ]]\
    .rename(columns=rename_mapping)

product_etl

,PRODUCT_name,PRODUCT_image,PRODUCT_id,PRODUCT_description,LANGUAGE_name,PRODUCT_PRODUCTION_COST_cost,PRODUCT_MARGIN_percentage,PRODUCT_INTRODUCTION_DATE_date,PRODUCT_LINE_code,PRODUCT_LINE_name
0,TrailChef Water Bag,P01CE1CG1.jpg,1,"Lightweight, collapsible bag to carry liquids ...",EN,4,.33,15-2-2011,1,Camping Equipment
1,TrailChef Utensils,P10CE1CG1.jpg,10,"Spoon, fork and knife set made of a light yet ...",EN,10,.4,15-2-2011,1,Camping Equipment
2,Insect Bite Relief,P100OP4FA17.jpg,100,The Insect Bite Relief helps the itching and s...,EN,3,.5,15-2-2011,4,Outdoor Protection
3,Hailstorm Steel Irons,P101GE5IR18.jpg,101,Iron is 17-4 stainless steel. Shafts are grap...,EN,305.54,.43,15-12-2019,5,Golf Equipment
4,Hailstorm Titanium Irons,P102GE5IR18.jpg,102,Made entirely of pure titanium. The ultimate i...,EN,380.95,.51,10-12-2019,5,Golf Equipment
...,...,...,...,...,...,...,...,...,...,...
110,Sun Shield,P91OP4SS16.jpg,95,"PABA free sunscreen, SPF 30, poison oak and iv...",EN,3,.5,15-2-2011,4,Outdoor Protection
111,Compact Relief Kit,P96OP4FA17.jpg,96,A personal first aid kit is recommended for ev...,EN,16.43,.28,15-2-2011,4,Outdoor Protection
112,Deluxe Family Relief Kit,P96OP4FA17.jpg,97,A complete medical kit suitable for families w...,EN,25,.28,5-3-2013,4,Outdoor Protection
113,Calamine Relief,P98OP4FA17.jpg,98,Use the Calamine Relief for allergic skin reac...,EN,3,.5,15-2-2011,4,Outdoor Protection


## Sales Staff ETL

## Satisfaction type ETL

In [428]:
satisfaction_type_etl = satisfaction_type[["SATISFACTION_TYPE_DESCRIPTION"]]\
        .rename(columns=rename_mapping)

satisfaction_type_etl

,SATISFACTION_TYPE_description
0,Not satisfied
1,Less than satisfied
2,Satisfied
3,Very Satisfied
4,More than satisfied


## Course ETL

In [429]:
course_etl = course[["COURSE_DESCRIPTION"]]\
        .rename(columns=rename_mapping)

course_etl

,COURSE_description
0,GO Orientation
1,GO Communication
2,GO Sales 1
3,GO Sales 2
4,GO Marketing 1
5,GO Marketing 2
6,GO Marketing 3
7,GO Management 1
8,GO Management 2


## Sales Forecast ETL

In [430]:
sales_forecast_etl = sales_forecast[["YEAR", "MONTH", "EXPECTED_VOLUME"]]\
        .rename(columns=rename_mapping)

sales_forecast_etl

,YEAR_number,MONTH_number,EXPECTED_VOLUME_volume
0,2022,12,383
1,2021,1,80
2,2021,2,51
3,2021,3,214
4,2021,4,300
...,...,...,...
3867,2022,8,282
3868,2022,9,920
3869,2022,10,1081
3870,2022,11,398


## Retailer Contact ETL

In [431]:
retailer_contact_etl = pd.merge(retailer_contact, retailer_site, on='RETAILER_SITE_CODE')
retailer_contact_etl = pd.merge(retailer_contact_etl, country, on='COUNTRY_CODE')
retailer_contact_etl = pd.merge(retailer_contact_etl, sales_territory, on='SALES_TERRITORY_CODE')\
    [['RETAILER_CONTACT_CODE', 'RETAILER_SITE_CODE', 'FIRST_NAME',
       'LAST_NAME', 'JOB_POSITION_EN', 'EXTENSION', 'FAX', 'E_MAIL', 'GENDER', 
       'RETAILER_CODE', 'ADDRESS1', 'ADDRESS2', 'CITY', 'REGION',
       'POSTAL_ZONE', 'COUNTRY_CODE', 'ACTIVE_INDICATOR', 
       'COUNTRY_EN', 'LANGUAGE', 'CURRENCY_NAME', 
       'FLAG_IMAGE', 'SALES_TERRITORY_CODE',
       'TERRITORY_NAME_EN']]\
    .rename(columns=rename_mapping)

retailer_contact_etl

,RETAILER_CONTACT_code,RETAILER_SITE_code,FIRST_NAME_name,LAST_NAME_name,JOB_POSITION_EN,EXTENSION_number,FAX_number,EMAIL_address,GENDER_code,RETAILER_code,...,REGION_name,POSTAL_ZONE_code,COUNTRY_code,ACTIVE_INDICATOR_bool,COUNTRY_name,LANGUAGE_name,CURRENCY_name,FLAG_image,SALES_TERRITORY_code,TERRITORY_name
0,10,14,Frank,Smith,Chief Purchaser,2489,1 (514) 741-1610,FSmith@legolfeurinc.com,M,93,...,Québec,H2Y 2W2,4,1,Canada,EN,dollars,F04,1,Americas
1,100,85,Amanda,Yates,Stock Manager,None,1 (702) 369-2274,Y2883@emertxe.com,F,104,...,Nevada,89118,3,1,United States,EN,dollars,F03,1,Americas
2,101,84,Paula,Schmidt,Assistant Purchaser,None,1 (702) 882-0978,S8832@emertxe.com,F,104,...,Nevada,89763,3,1,United States,EN,dollars,F03,1,Americas
3,102,88,Micheal,Tao,District Manager,445,1 (713) 524-3215,T8839@emertxe.com,M,104,...,Texas,77112,3,1,United States,EN,dollars,F03,1,Americas
4,103,92,Jack,Murphy,District Manager,None,1 (305) 557-5478,Murphy@eyedimensions3.com,M,105,...,Florida,33021,3,0,United States,EN,dollars,F03,1,Americas
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
386,95,87,Jennifer,Gomes,Site Manager,None,1 (423) 266-4721,G1289@emertxe.com,F,104,...,Tennessee,37405-0955,3,1,United States,EN,dollars,F03,1,Americas
387,96,91,Timothy,Moon,Site Assistant Manager,None,1 (304) 343-4560,M0019@emertxe.com,M,104,...,West Virginia,25301-1299,3,1,United States,EN,dollars,F03,1,Americas
388,97,90,Paul,Fischer,Stock Manager,None,1 (206) 362-6645,F9943@emertxe.com,M,104,...,Washington,98154,3,1,United States,EN,dollars,F03,1,Americas
389,98,89,Derek,Wilkes,Site Assistant Manager,None,1 (703) 549-4896,W3219@emertxe.com,M,104,...,Virginia,22323,3,1,United States,EN,dollars,F03,1,Americas


## Retailer ETL

In [432]:
retailer_etl = pd.merge(retailer, retailer_headquarters, on='RETAILER_CODEMR')
retailer_etl = pd.merge(retailer_etl, retailer_type, on='RETAILER_TYPE_CODE')
retailer_etl = pd.merge(retailer_etl, retailer_segment, on='SEGMENT_CODE').rename(columns={'LANGUAGE':'SEGMENT_LANGUAGE_code'})

retailer_etl = pd.merge(retailer_etl, country, on='COUNTRY_CODE').rename(columns={'LANGUAGE':'COUNTRY_LANGUAGE_code'})
retailer_etl = retailer_etl[retailer_etl.columns.difference(['TRIAL219','TRIAL222_x','TRIAL222_y','TRIAL222'])]

retailer_etl = pd.merge(retailer_etl, sales_territory, on='SALES_TERRITORY_CODE')\
    [['RETAILER_CODE', 'RETAILER_CODEMR', 'COMPANY_NAME',
       'RETAILER_TYPE_CODE', 'RETAILER_NAME', 'ADDRESS1',
       'ADDRESS2', 'CITY', 'REGION', 'POSTAL_ZONE', 'COUNTRY_CODE', 'PHONE',
       'FAX', 'SEGMENT_CODE', 'RETAILER_TYPE_EN', 'SEGMENT_LANGUAGE_code',
       'COUNTRY_EN', 'COUNTRY_LANGUAGE_code', 'CURRENCY_NAME', 'FLAG_IMAGE',
       'SALES_TERRITORY_CODE', 'TERRITORY_NAME_EN']]\
    .rename(columns=rename_mapping)

retailer_etl.columns

Index(['RETAILER_code', 'RETAILER_MR_code', 'COMPANY_name',
       'RETAILER_TYPE_code', 'RETAILER_name', 'ADDRESS1_address',
       'ADDRESS2_address', 'CITY_name', 'REGION_name', 'POSTAL_ZONE_code',
       'COUNTRY_code', 'PHONE_number', 'FAX_number', 'SEGMENT_code',
       'RETAILER_TYPE_name', 'SEGMENT_LANGUAGE_code', 'COUNTRY_name',
       'COUNTRY_LANGUAGE_code', 'CURRENCY_name', 'FLAG_image',
       'SALES_TERRITORY_code', 'TERRITORY_name'],
      dtype='object')

## Order ETL

In [433]:
order_etl = pd.merge(order_header, order_method, on='ORDER_METHOD_CODE')\
    [['ORDER_NUMBER', 'RETAILER_NAME',
       'RETAILER_CONTACT_CODE', 'SALES_STAFF_CODE',
       'ORDER_DATE', 'ORDER_METHOD_CODE', 'ORDER_METHOD_EN']]\
    .rename(columns=rename_mapping)

order_etl

,ORDER_number,RETAILER_name,RETAILER_CONTACT_code,SALES_STAFF_code,ORDER_DATE_date,ORDER_METHOD_code,ORDER_METHOD_name
0,1153,Ultra Sports,6,50,2020-04-14,7,Sales visit
1,1154,Ultra Sports,6,50,2020-10-01,4,E-mail
2,1155,Ultra Sports,65,49,2020-04-21,7,Sales visit
3,1156,Ultra Sports,65,49,2020-09-09,5,Web
4,1157,Ultra Sports,66,50,2020-04-10,2,Telephone
...,...,...,...,...,...,...,...
5355,9475,Preben's T°y,329,29,2020-01-13,4,E-mail
5356,9476,Preben's T°y,327,29,2021-01-02,5,Web
5357,9477,Preben's T°y,329,29,2021-01-18,2,Telephone
5358,9478,Preben's T°y,327,27,2022-01-15,7,Sales visit


## Return reason ETL

In [434]:
return_reason_etl = return_reason[["RETURN_DESCRIPTION_EN"]]\
        .rename(columns=rename_mapping)

return_reason_etl

,RETURN_REASON_description
0,Defective product
1,Incomplete product
2,Wrong product ordered
3,Wrong product shipped
4,Unsatisfactory product


## Returned Item ETL

In [435]:
# Rename columns
returned_item_etl = returned_item.rename(columns=rename_mapping)
# Remove unnecessary columns
returned_item_etl = returned_item_etl[returned_item_etl.columns.difference(['TRIAL888'])]

returned_item_etl

,ORDER_DETAIL_code,RETURN_DATE_date,RETURN_QUANTITY_number,RETURN_REASON_code,RETURN_code
0,84858,1-8-2021 04:10:24,8,5,1491
1,84440,6-12-2020 18:46:19,2,2,1492
2,84867,24-6-2021 09:23:14,22,4,1493
3,84873,21-7-2021 00:00:09,20,3,1494
4,84488,7-10-2020 05:13:58,2,1,1496
...,...,...,...,...,...
701,114972,6-6-2021 09:22:12,24,3,2465
702,114983,6-10-2021 12:31:25,66,5,2466
703,115215,31-3-2022 15:39:38,64,4,2467
704,115171,23-12-2022 19:48:50,2,1,2468


## Order Details ETL

In [437]:
# Rename columns
order_detail_etl = order_details.rename(columns=rename_mapping)
# Remove unnecessary columns
order_detail_etl = order_detail_etl[order_detail_etl.columns.difference(['TRIAL879'])]

order_detail_etl


,ORDER_DETAIL_code,ORDER_number,PRODUCT_id,QUANTITY_number,UNIT_COST_dollar,UNIT_PRICE_dollar,UNIT_SALE_PRICE_dollar
0,100000,8462,96,16,16.1,22.54,22.54
1,100001,9111,96,20,16.1,22.54,22.54
2,100002,8451,96,24,16.1,22.54,22.54
3,100003,8453,96,18,16.1,22.54,22.54
4,100004,8439,96,20,16.1,22.54,22.54
...,...,...,...,...,...,...,...
43058,99995,8459,95,146,2.76,5.55,5.55
43059,99996,8457,95,172,2.76,5.55,5.55
43060,99997,9267,95,192,2.76,5.55,5.55
43061,99998,8441,95,192,2.76,5.55,5.55
